In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ujson as json
import jsonlines
import os,sys
sys.path.append('/dfs/scratch1/simran/tutorial/bootleg-internal/notebooks')
sys.path.append('/dfs/scratch1/simran/tutorial/bootleg-internal/')
import time
import multiprocessing
from tabulate import tabulate
from collections import defaultdict
import os,sys
import pandas as pd
import jsonlines
from tqdm import tqdm
from collections import defaultdict
import nltk
nltk.download('stopwords')
import utils_not_for_public as du

/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /lfs/1/simran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
pd.set_option('display.max_colwidth', -1)


In [4]:
import ujson
def load_json_file(filename):
    with open(filename, 'r') as f:
        contents = ujson.load(f)
    return contents

In [5]:
LABEL_TO_ID = {'no_relation': 0, 'per:title': 1, 'org:top_members/employees': 2, 'per:employee_of': 3, 
               'org:alternate_names': 4, 'org:country_of_headquarters': 5, 'per:countries_of_residence': 6, 
               'org:city_of_headquarters': 7, 'per:cities_of_residence': 8, 'per:age': 9, 
               'per:stateorprovinces_of_residence': 10, 'per:origin': 11, 'org:subsidiaries': 12, 
               'org:parents': 13, 'per:spouse': 14, 'org:stateorprovince_of_headquarters': 15, 'per:children': 16, 
               'per:other_family': 17, 'per:alternate_names': 18, 'org:members': 19, 'per:siblings': 20, 
               'per:schools_attended': 21, 'per:parents': 22, 'per:date_of_death': 23, 'org:member_of': 24, 
               'org:founded_by': 25, 'org:website': 26, 'per:cause_of_death': 27, 
               'org:political/religious_affiliation': 28, 'org:founded': 29, 'per:city_of_death': 30, 
               'org:shareholders': 31, 'org:number_of_employees/members': 32, 'per:date_of_birth': 33, 
               'per:city_of_birth': 34, 'per:charges': 35, 'per:stateorprovince_of_death': 36, 'per:religion': 37, 
               'per:stateorprovince_of_birth': 38, 'per:country_of_birth': 39, 'org:dissolved': 40, 
               'per:country_of_death': 41}

LABEL_LST = list(LABEL_TO_ID.keys())

STANFORD_NER_TYPES = ['DATE', 'LOCATION', 'MONEY', 'ORGANIZATION', 'PERCENT', 'PERSON', 'TIME']
punctuation = [',', ':', '.', ';', "'", ')', '(', "'s", '--', '-', '``', "''"]

In [6]:
base_data = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/tacred/base_data'

train_file = "{}/train.json".format(base_data)
with open(train_file) as train:
    df_train = json.load(train)
    df_train = pd.DataFrame.from_dict(df_train, orient='columns')
    print(df_train.shape)
    
dev_file = "{}/dev_rev.json".format(base_data)
with open(dev_file) as dev:
    df_dev = json.load(dev)
    df_dev = pd.DataFrame.from_dict(df_dev, orient='columns')
    print(df_dev.shape)
    
test_file = "{}/test_rev.json".format(base_data)
with open(test_file) as test:
    df_test = json.load(test)
    df_test = pd.DataFrame.from_dict(df_test, orient='columns')
    print(df_test.shape)

(68124, 14)
(22631, 14)
(15509, 14)


In [7]:
train_ids = df_train.loc[:,'id'].tolist()
dev_ids = df_dev.loc[:,'id'].tolist()
test_ids = df_test.loc[:,'id'].tolist()

In [8]:
def load_rel_names():
    return json.load(open("/dfs/scratch1/simran/bootleg_downstream/archive/wikidata_mappings/pid_names.json"))

es, esp, type_vocab, typeid2typename, type_vocab_wd, typeid2typename_wd, alias_qid_count = du.load_esp("wiki", kg_adj='kg_adj_complete.txt',
kg_triples='kg_triples.txt',
hy_vocab='hyena_vocab.json',
hy_types='hyena_types_wiki_all_aug.json',
wd_vocab='wikidata_to_typeid.json',
wd_types='wikidata_types_wiki_filt.json', overwrite=False) 

typename2title_wd = load_json_file("/dfs/scratch0/lorr1/bootleg/embs/wikidatatypeid_to_title.json")
typeqid_to_id = json.load(open("/dfs/scratch0/lorr1/bootleg/embs/wikidata_to_typeid.json"))
rel2id = '/dfs/scratch0/lorr1/bootleg/embs/relation_to_typeid.json'
rel_to_name = load_rel_names()

def get_relations_lst(qid1, qid2):
    # If there are multiple relationships between two qids (both directions)
    rels = [esp.get_relation_name(r) for r in esp.get_all_relations(qid1, qid2)]
    rels = [rel_to_name.get(rel, rel) for rel in rels]
    if rels:
        return rels[0]
    else:
        return ""

Reading in existing entity word bags from /dfs/scratch0/lorr1/data_prep/data/wiki0516/entity_db/entity_mappings/entity_bag_of_words
Reading in vocab from /dfs/scratch0/lorr1/data_prep/data/wiki0516/entity_db/entity_mappings/entity_all_words/all_words_vocab.marisa
Loaded entity symbols.
Loading alias qid counts from /dfs/scratch0/lorr1/data_prep/data/wiki0516/stats/alias_qid_traindata_withaugment.json
Found 506 hyena types
Found 67110 wikidata types
FINISHED LOADING IN 95.12413239479065


In [9]:
def load_rel_names_0905():
    return json.load(open("/dfs/scratch1/simran/bootleg_downstream/archive/wikidata_mappings/pid_names.json"))

es_0905, esp_0905, type_vocab_0905, typeid2typename_0905, type_vocab_wd_0905, typeid2typename_wd_0905, alias_qid_count_0905 = du.load_esp("wiki_0905", overwrite=False)     

typename2title_wd_0905 = load_json_file("/dfs/scratch0/lorr1/bootleg/embs/wikidatatypeid_to_title_0905.json")
typeqid_to_id_0905 = json.load(open("/dfs/scratch0/lorr1/bootleg/embs/wikidata_to_typeid_0905.json"))
rel2id_0905 = '/dfs/scratch0/lorr1/bootleg/embs/relation_to_typeid_0905.json'
rel_to_name_0905 = load_rel_names_0905()

def get_relations_lst_0905(qid1, qid2):
    # If there are multiple relationships between two qids (both directions)
    rels = [esp_0905.get_relation_name(r) for r in esp_0905.get_all_relations(qid1, qid2)]
    rels = [rel_to_name_0905.get(rel, rel) for rel in rels]
    if rels:
        return rels[0]
    else:
        return ""

Reading in existing entity word bags from /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings/entity_bag_of_words
Reading in vocab from /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings/entity_all_words/all_words_vocab.marisa
Loaded entity symbols.
Loading alias qid counts from /dfs/scratch0/lorr1/data_prep/data/wiki0905/stats/alias_qid_traindata_withaugment.json
Found 506 hyena types
Found 26933 wikidata types
FINISHED LOADING IN 93.9799337387085


In [10]:
def load_mentions(file): 
    lines = []
    with jsonlines.open(file) as f: 
        for line in f: 
            new_line = {
                'id': line['id'],
                'sentence': line['sentence'],
                'aliases': line['aliases'], 
                'spans': line['spans'],
                'qids': line['qids'],
                #'anchor': line['anchor'],
                'sent_idx_unq': line['sent_idx_unq'],
                'probs': line['probs'], # probability of gold  ( value )
                'ctx_emb_ids': line['ctx_emb_ids'],
                'entity_ids': line['entity_ids'] 
                # candprobs # probability for all candidates (N x k matrix)
            }
            lines.append(new_line)
    return pd.DataFrame(lines)

### Load the results

In [11]:
model_iclr = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_092620/basic_full_sentences/static_remap_embs/09262020-234021_posaware_test_rev_ent.json.csv' 
spanboot_iclr = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_092620/basic_full_sentences/predictions/test_predictions.txt'

model_cidr = '/dfs/scratch1/simran/bootleg_downstream/error_outputs/baseline_bootleg_test_rev_ent.csv'
spanboot_cidr = '/dfs/scratch1/simran/bootleg_downstream/error_outputs/0925_spanboot_test_predictions.csv'


In [12]:
df_cidr = pd.read_csv(model_cidr)
df_results_cidr = pd.read_csv(spanboot_cidr)
df_cidr['prediction'] = df_results_cidr['prediction']
df_cidr_errors = df_cidr[df_cidr['relation'] != df_cidr['prediction']]

In [13]:
df_iclr = pd.read_csv(model_iclr)
df_results_iclr = pd.read_csv(spanboot_iclr, sep="\t", header=None)
df_results_iclr.columns = ["id","prediction"]
df_iclr['prediction'] = df_results_iclr['prediction']
df_iclr_errors = df_iclr[df_iclr['relation'] != df_iclr['prediction']]

In [14]:
print(df_cidr_errors.shape[0])
print(df_iclr_errors.shape[0])

1124
1231


In [43]:
cidr_error_ids = df_cidr_errors['id'].tolist()
iclr_error_ids = df_iclr_errors['id'].tolist()

# load iclr data

In [15]:
base_directory_0905 = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_092620/'
expt_directory_0905 = 'basic_full_sentences/'
boot_labels_file_0905 = "{}{}bootleg_labels.jsonl".format(base_directory_0905, expt_directory_0905)
bootleg_labels_df_0905 = load_mentions(boot_labels_file_0905)
print(bootleg_labels_df_0905.columns)
print(bootleg_labels_df_0905.shape)

Index(['id', 'sentence', 'aliases', 'spans', 'qids', 'sent_idx_unq', 'probs',
       'ctx_emb_ids', 'entity_ids'],
      dtype='object')
(106264, 9)


# load cidr data

In [16]:
base_directory = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_cidr_model/bootleg_09132020/'
expt_directory = 'basic_full_sentences/'
boot_labels_file = "{}{}bootleg_labels.jsonl".format(base_directory, expt_directory)
bootleg_labels_df = load_mentions(boot_labels_file)
print(bootleg_labels_df.columns)
print(bootleg_labels_df.shape)

Index(['id', 'sentence', 'aliases', 'spans', 'qids', 'sent_idx_unq', 'probs',
       'ctx_emb_ids', 'entity_ids'],
      dtype='object')
(106264, 9)


# save unique QIDs for mini_bootleg_experiment

### save slices of tacred for fine-tune experiment

In [51]:
ERROR_COUNT = 2000
CORRECT_COUNT = 20

In [53]:
# ERROR QIDS
import ast
# error_qids_cidr = []
# cand_qids_cidr = []
# count = ERROR_COUNT
# for ind, row in tqdm(df_cidr_errors.iterrows()):
#     lst = ast.literal_eval(row['qids'])
    
#     bootleg_row = bootleg_labels_df[bootleg_labels_df['id'] == row['id']]
#     alias_lst = bootleg_row['aliases'].item()
#     [cand_qids_cidr.extend(esp.get_qid_cands(alias)) for alias in alias_lst[0:min(3,len(alias_lst))]]
#     error_qids_cidr.extend(lst)
#     if count == 0:
#         break
#     count -= 1
# print(len(set(error_qids_cidr)))
print(Counter(error_qids_cidr).most_common(100))

0it [00:00, ?it/s]/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: `item` has been deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.
1124it [00:08, 132.35it/s]

2546
[('UNK', 28260), ('Q1186190', 92), ('Q148', 81), ('Q2498389', 72), ('Q30', 66), ('Q179202', 65), ('Q736402', 65), ('Q607131', 60), ('Q943091', 60), ('Q212235', 59), ('Q459256', 56), ('Q7927961', 52), ('Q60', 51), ('Q4739665', 48), ('Q20711120', 48), ('Q487907', 46), ('Q1137260', 44), ('Q4379875', 44), ('Q11725', 43), ('Q241', 43), ('Q6011891', 42), ('Q65', 42), ('Q625657', 40), ('Q1967040', 40), ('Q6978831', 40), ('Q4466010', 39), ('Q3161542', 39), ('Q7039832', 36), ('Q6990032', 34), ('Q930069', 34), ('Q219569', 33), ('Q115860', 33), ('Q217072', 32), ('Q463416', 32), ('Q380767', 30), ('Q38', 30), ('Q1888308', 29), ('Q1458155', 28), ('Q192283', 28), ('Q77617', 28), ('Q843', 28), ('Q884', 27), ('Q7151312', 26), ('Q1152465', 26), ('Q777636', 26), ('Q26465818', 26), ('Q2655853', 26), ('Q230673', 26), ('Q79718', 26), ('Q796', 25), ('Q48537', 25), ('Q5987345', 24), ('Q20709176', 24), ('Q48525', 24), ('Q351280', 24), ('Q5071844', 24), ('Q6978094', 24), ('Q145', 23), ('Q61', 23), ('Q19295

In [ ]:
most_common_error_cidr = Counter(error_qids_cidr).most_common(100)
for tup in most_common_error_cidr:
    try:
        print(es.get_title(tup[0]))
    except:
        pass

In [54]:
error_qids_iclr = []
cand_qids_iclr = []
count = ERROR_COUNT
for ind, row in tqdm(df_iclr_errors.iterrows()):
    lst = ast.literal_eval(row['qids'])
    
    bootleg_row = bootleg_labels_df_0905[bootleg_labels_df_0905['id'] == row['id']]
    alias_lst = bootleg_row['aliases'].item()
    [cand_qids_iclr.extend(esp_0905.get_qid_cands(alias)) for alias in alias_lst[0:min(3,len(alias_lst))]]
    error_qids_iclr.extend(lst)
    if count == 0:
        break
    count -= 1
print(len(set(error_qids_iclr)))

0it [00:00, ?it/s]/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: `item` has been deprecated and will be removed in a future version
  
1231it [00:10, 115.86it/s]

2924


In [ ]:
most_common_error_iclr = Counter(error_qids_iclr).most_common(100)
for tup in most_common_error_iclr:
    try:
        print(es.get_title(tup[0]))
    except:
        pass

In [16]:
print("QIDs in examples CIDR has errors on:",len(set( error_qids_cidr)))
print("QIDs in examples ICLR has errors on:",len(set( error_qids_iclr)))
print("CANDS for all QIDs in examples CIDR has errors on:", len(set(cand_qids_cidr)))
print("CANDS for all QIDs in examples ICLR has errors on:",len(set( cand_qids_iclr)))

superset_errors = error_qids_cidr
superset_errors.extend(error_qids_iclr)
print("Superset of QIDs in examples CIDR+ICLR has errors on:", len(set(superset_errors)))

superset_errors_cands = cand_qids_cidr
superset_errors_cands.extend(cand_qids_iclr)
print("Superset of CANDS for examples CIDR+ICLR has errors on:", len(set(superset_errors_cands)))

QIDs in examples CIDR has errors on: 114
QIDs in examples ICLR has errors on: 132
CANDS for all QIDs in examples CIDR has errors on: 739
CANDS for all QIDs in examples ICLR has errors on: 849
Superset of QIDs in examples CIDR+ICLR has errors on: 197
Superset of CANDS for examples CIDR+ICLR has errors on: 1170


In [17]:
# Correct QIDS
count = CORRECT_COUNT
correct_qids_cidr = []
correct_qids_cands_cidr = []
for ind, row in tqdm(bootleg_labels_df_0905.iterrows()):
    correct_qids_cidr.extend(row['qids'])
    alias_lst = row['aliases']
    [correct_qids_cands_cidr.extend(esp_0905.get_qid_cands(alias)) for alias in alias_lst[0:min(3,len(alias_lst))]]
    
    if count == 0:
        break
    count -= 1
print(len(set(correct_qids_cidr)))

20it [00:00, 111.57it/s]

145


In [18]:
count = CORRECT_COUNT
correct_qids_iclr = []
correct_qids_cands_iclr = []
for ind, row in tqdm(bootleg_labels_df.iterrows()):
    correct_qids_iclr.extend(row['qids'])
    alias_lst = row['aliases']
    [correct_qids_cands_iclr.extend(esp.get_qid_cands(alias)) for alias in alias_lst[0:min(3,len(alias_lst))]]
    
    if count == 0:
        break
    count -= 1
print(len(set(correct_qids_iclr)))

20it [00:00, 151.93it/s]

130


In [19]:
print("QIDs in examples CIDR:",len(set( correct_qids_cidr)))
print("QIDs in examples ICLR:",len(set( correct_qids_iclr)))
print("CANDS for all QIDs in examples CIDR:", len(set(correct_qids_cands_cidr)))
print("CANDS for all QIDs in examples ICLR:",len(set( correct_qids_cands_iclr)))

superset_correct = correct_qids_cidr
superset_correct.extend(correct_qids_iclr)
print("Superset of QIDs in examples CIDR+ICLR:", len(set(superset_correct)))

superset_correct_cands = correct_qids_cands_cidr
superset_correct_cands.extend(correct_qids_cands_iclr)
print("Superset of CANDS for examples CIDR+ICLR:", len(set(superset_correct_cands)))

QIDs in examples CIDR: 145
QIDs in examples ICLR: 130
CANDS for all QIDs in examples CIDR: 1029
CANDS for all QIDs in examples ICLR: 977
Superset of QIDs in examples CIDR+ICLR: 181
Superset of CANDS for examples CIDR+ICLR: 1185


In [20]:
overall_qids = []
overall_qids = superset_errors
overall_qids.extend(superset_correct)
overall_qids.extend(superset_correct_cands[0:350])
overall_qids.extend(superset_errors_cands[0:350])

print(len(set(overall_qids)))

1011


In [21]:
# with open('../bootleg_092620/boot_info_for_laurel/tacred_qids_iclr_and_cidr_2.txt', 'w') as f:
#     for item in set(overall_qids):
#         f.write("%s\n" % item)

## load experiment QIDs for error analysis of mini experiments

In [17]:
# load all the predictions
base_for_preds = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations'

model_ft = '{}/ft_expts/basic_full_sentences/10122020-191913__test_rev_ent.json.csv'.format(base_for_preds)
model_full = '{}/full_expts/basic_full_sentences/10092020-200033__test_rev_ent.json.csv'.format(base_for_preds)
model_clean = '{}/clean_expts/basic_full_sentences/10102020-010600__test_rev_ent.json.csv'.format(base_for_preds)
model_clean_men = '{}/clean_men/basic_full_sentences/10092020-230622__test_rev_ent.json.csv'.format(base_for_preds)
model_noty = '{}/noty_expts/basic_full_sentences/10122020-173117__test_rev_ent.json.csv'.format(base_for_preds)
model_cidr = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_cidr_model/bootleg_09132020/basic_full_sentences/10122020-223113_static_remap_embs_test_rev_ent.json.csv'

# model_ft_bl = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_model_1002/experiments_ft/basic_full_sentences9/10122020-231645__test_rev_ent.json.csv'
# model_noft_bl = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_model_1002/experiments_noft/basic_full_sentences9/10122020-231501__test_rev_ent.json.csv'
model_ft_bl = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_model_1002/experiments_ft/basic_full_sentences/10122020-233525__test_rev_ent.json.csv'
model_noft_bl = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_model_1002/experiments_noft/basic_full_sentences/10132020-002014__test_rev_ent.json.csv'


model_ft_df = pd.read_csv(model_ft)
model_full_df = pd.read_csv(model_full)
model_clean_df = pd.read_csv(model_clean)
model_clean_men_df = pd.read_csv(model_clean_men)
model_noty_df = pd.read_csv(model_noty)
model_cidr_df = pd.read_csv(model_cidr)
model_ft_bl_df =  pd.read_csv(model_ft_bl)
model_noft_bl_df =  pd.read_csv(model_noft_bl)


In [18]:
model_full_id = {}
model_clean_men_id = {}
model_noty_id = {}
model_ft_id = {}
model_cidr_id = {}
model_ft_bl_id = {}
model_noft_bl_id = {}

for ind, row in model_full_df.iterrows():
    model_full_id[row['id']] = row['prediction'] 
    
for ind, row in model_ft_df.iterrows():
    model_ft_id[row['id']] = row['prediction'] 
    
for ind, row in model_clean_men_df.iterrows():
    model_clean_men_id[row['id']] = row['prediction'] 
    
for ind, row in model_noty_df.iterrows():
    model_noty_id[row['id']] = row['prediction'] 
    
for ind, row in model_cidr_df.iterrows():
    model_cidr_id[row['id']] = row['prediction'] 
    
for ind, row in model_ft_bl_df.iterrows():
    model_ft_bl_id[row['id']] = row['prediction'] 
    
for ind, row in model_noft_bl_df.iterrows():
    model_noft_bl_id[row['id']] = row['prediction'] 
    
model_clean_df['prediction_full'] = model_clean_df['id'].map(model_full_id)
model_clean_df['prediction_clean_men'] = model_clean_df['id'].map(model_clean_men_id)
model_clean_df['prediction_noty'] = model_clean_df['id'].map(model_noty_id)
model_clean_df['prediction_ft'] = model_clean_df['id'].map(model_ft_id)
model_clean_df['prediction_cidr'] = model_clean_df['id'].map(model_cidr_id)
model_clean_df['prediction_ft_bl'] = model_clean_df['id'].map(model_ft_bl_id)
model_clean_df['prediction_noft_bl'] = model_clean_df['id'].map(model_noft_bl_id)
model_clean_df.rename(columns={'prediction':'prediction_clean'}, inplace=True)


In [19]:
model_clean_df.columns

Index(['obj_type', 'example', 'obj_leng', 'qids', 'real_mentions', 'obj', 'id',
       'subj_leng', 'subj_qids', 'separation_dist', 'obj_mentions', 'mentions',
       'subj_mentions', 'num_ner', 'obj_ner', 'stanford_ner', 'subj',
       'prediction_clean', 'obj_pos', 'prop_mentions', 'subj_type', 'relation',
       'prop_ner', 'subj_pos', 'obj_qids', 'subj_ner', 'prediction_full',
       'prediction_clean_men', 'prediction_noty', 'prediction_ft',
       'prediction_cidr', 'prediction_ft_bl', 'prediction_noft_bl'],
      dtype='object')

In [20]:
full_count = model_clean_df[model_clean_df['relation'] != model_clean_df['prediction_full']].shape[0]
clean_men_count = model_clean_df[model_clean_df['relation'] != model_clean_df['prediction_clean_men']].shape[0]
clean_count = model_clean_df[model_clean_df['relation'] != model_clean_df['prediction_clean']].shape[0]
noty_count = model_clean_df[model_clean_df['relation'] != model_clean_df['prediction_noty']].shape[0]
ft_count = model_clean_df[model_clean_df['relation'] != model_clean_df['prediction_ft']].shape[0]
cidr_count = model_clean_df[model_clean_df['relation'] != model_clean_df['prediction_cidr']].shape[0]
ft_bl = model_clean_df[model_clean_df['relation'] != model_clean_df['prediction_ft_bl']].shape[0]
noft_bl = model_clean_df[model_clean_df['relation'] != model_clean_df['prediction_noft_bl']].shape[0]
        
print("full={}, clean_men={}, clean={}, noty={}, ft={} cidr={} ft_bl={}, noft_bl={}\n".format(full_count, clean_men_count, clean_count, noty_count, ft_count, cidr_count, ft_bl, noft_bl))
print(ft_count/model_clean_df.shape[0])
print(cidr_count/model_clean_df.shape[0])


full=1636, clean_men=1663, clean=1614, noty=1614, ft=1572 cidr=1569 ft_bl=1673, noft_bl=1595

0.10136050035463279
0.10116706428525372


In [21]:
#error_rate_by_relation(df_results, 'relation_model', df_errors){
relation_res = {}

for k, v in LABEL_TO_ID.items():
    if model_clean_df[model_clean_df['relation'] == k].shape[0] > 0:
        sub_df = model_clean_df[model_clean_df['relation'] == k]
        full_count = sub_df[sub_df['relation'] != sub_df['prediction_full']].shape[0]
        clean_men_count = sub_df[sub_df['relation'] != sub_df['prediction_clean_men']].shape[0]
        clean_count = sub_df[sub_df['relation'] != sub_df['prediction_clean']].shape[0]
        noty_count = sub_df[sub_df['relation'] != sub_df['prediction_noty']].shape[0]
        ft_count = sub_df[sub_df['relation'] != sub_df['prediction_ft']].shape[0]
        cidr_count = sub_df[sub_df['relation'] != sub_df['prediction_cidr']].shape[0]
        ft_bl = sub_df[sub_df['relation'] != sub_df['prediction_ft_bl']].shape[0]
        noft_bl = sub_df[sub_df['relation'] != sub_df['prediction_noft_bl']].shape[0]
        
        print("{}: full={}, clean_men={}, clean={}, noty={}, ft={}, cidr={}, ft_bl={}, noft_bl={}".format(k, full_count, clean_men_count, clean_count, noty_count, ft_count, cidr_count, ft_bl, noft_bl))
        relation_res[k] = {'full':full_count, 'clean_men':clean_men_count, 'clean':clean_count, 'noty':noty_count, 'ft':ft_count, 'cidr':cidr_count, 'ft_bl':ft_bl, 'noft_bl':noft_bl}
                           
                           

no_relation: full=732, clean_men=864, clean=655, noty=673, ft=676, cidr=755, ft_bl=772, noft_bl=702
per:title: full=73, clean_men=42, clean=48, noty=96, ft=48, cidr=58, ft_bl=74, noft_bl=54
org:top_members/employees: full=32, clean_men=27, clean=25, noty=27, ft=33, cidr=21, ft_bl=38, noft_bl=29
per:employee_of: full=61, clean_men=62, clean=66, noty=63, ft=67, cidr=67, ft_bl=61, noft_bl=53
org:alternate_names: full=50, clean_men=48, clean=45, noty=44, ft=48, cidr=40, ft_bl=46, noft_bl=43
org:country_of_headquarters: full=60, clean_men=50, clean=55, noty=56, ft=52, cidr=51, ft_bl=55, noft_bl=57
per:countries_of_residence: full=57, clean_men=41, clean=61, noty=61, ft=42, cidr=51, ft_bl=62, noft_bl=58
org:city_of_headquarters: full=31, clean_men=35, clean=27, noty=27, ft=26, cidr=24, ft_bl=29, noft_bl=29
per:cities_of_residence: full=55, clean_men=34, clean=49, noty=36, ft=43, cidr=36, ft_bl=48, noft_bl=55
per:age: full=13, clean_men=14, clean=18, noty=17, ft=18, cidr=9, ft_bl=16, noft_bl=

In [22]:
print("{:<50} {:<10} {:<10} {:<10}".format('Key','FT','CIDR', 'NOFT_BL'))
for k, v in relation_res.items():
    ft = v['ft']
    cidr = v['cidr']
    noft_bl = v['noft_bl']
    print("{:<50} {:<10} {:<10} {:<10}".format(k, ft, cidr, noft_bl))

Key                                                FT         CIDR       NOFT_BL   
no_relation                                        676        755        702       
per:title                                          48         58         54        
org:top_members/employees                          33         21         29        
per:employee_of                                    67         67         53        
org:alternate_names                                48         40         43        
org:country_of_headquarters                        52         51         57        
per:countries_of_residence                         42         51         58        
org:city_of_headquarters                           26         24         29        
per:cities_of_residence                            43         36         55        
per:age                                            18         9          11        
per:stateorprovinces_of_residence                  18         16         25 

In [28]:
import ast

file_qids = open('../bootleg_092620/boot_info_for_laurel/tacred_qids_iclr_and_cidr.txt', 'r')
qids_expt = file_qids.read().split('\n')
qids_expt.remove('UNK')
inds_w_qid = []

for ind, row in tqdm(model_cidr_df.iterrows()):
    qids_lst = []
#     qids_lst = ast.literal_eval(row['subj_qids'])
#     qids_lst.extend(ast.literal_eval(row['obj_qids']))
    qids_lst = ast.literal_eval(row['qids'])
    
    if any(qid in qids_lst for qid in qids_expt):
        inds_w_qid.append(row['id'])

print(len(inds_w_qid))

15509it [00:08, 1832.86it/s]

5365


In [29]:
sub_df_expt = model_clean_df[model_clean_df['id'].isin(inds_w_qid)]
full_count = sub_df_expt[sub_df_expt['relation'] != sub_df_expt['prediction_full']].shape[0]
clean_men_count = sub_df_expt[sub_df_expt['relation'] != sub_df_expt['prediction_clean_men']].shape[0]
clean_count = sub_df_expt[sub_df_expt['relation'] != sub_df_expt['prediction_clean']].shape[0]
noty_count = sub_df_expt[sub_df_expt['relation'] != sub_df_expt['prediction_noty']].shape[0]
ft_count = sub_df_expt[sub_df_expt['relation'] != sub_df_expt['prediction_ft']].shape[0]
cidr_count = sub_df_expt[sub_df_expt['relation'] != sub_df_expt['prediction_cidr']].shape[0]
ft_bl = sub_df_expt[sub_df_expt['relation'] != sub_df_expt['prediction_ft_bl']].shape[0]
noft_bl = sub_df_expt[sub_df_expt['relation'] != sub_df_expt['prediction_noft_bl']].shape[0]
        
print("full={}, clean_men={}, clean={}, noty={}, ft={}, cidr={}, ft_bl={}, noft_bl={} \n".format(full_count, clean_men_count, clean_count, noty_count, ft_count, cidr_count, ft_bl, noft_bl))
print(ft_count/len(inds_w_qid))
print(cidr_count/len(inds_w_qid))


full=661, clean_men=648, clean=658, noty=651, ft=615, cidr=621, ft_bl=620, noft_bl=624 

0.11463187325256291
0.1157502329916123


In [25]:
ft_qids_expt_df = sub_df_expt[sub_df_expt['relation'] != sub_df_expt['prediction_ft']]
ft_qids_expt_df_ids = ft_qids_expt_df['id'].tolist()

ft_qids_expt_df = sub_df_expt[sub_df_expt['relation'] != sub_df_expt['prediction_cidr']]
ft_qids_expt_df_ids = ft_qids_expt_df['id'].tolist()


# compare the qids assigned

In [26]:
def get_span(str_span):
    ss, se = str_span.split(":")
    return [int(ss), int(se)]

In [80]:
mismatch_idx = []
swapped_qids = []
all_iclr_qid = []
all_cidr_qid = []

for ind, row_cidr in tqdm(bootleg_labels_df.iterrows()):
    id_row = row_cidr['id']
    row_iclr = bootleg_labels_df_0905[bootleg_labels_df_0905['id']==id_row]
    
    not_in_iclr = [qid_cidr for qid_cidr in row_cidr['qids'] if qid_cidr not in row_iclr['qids'].item()]
    not_in_cidr = [qid_iclr for qid_iclr in row_iclr['qids'].item() if qid_iclr not in row_cidr['qids']]
    
    not_in_iclr_spans = [get_span(row_cidr['spans'][i]) for i in range(len(row_cidr['spans'])) if get_span(row_cidr['spans'][i]) not in row_iclr['spans'].item()]
    not_in_cidr_spans = [row_iclr['spans'].item()[i] for i in range(len(row_iclr['spans'].item())) if row_iclr['spans'].item()[i] not in row_cidr['spans']]
    
    
    if not_in_iclr or not_in_cidr:
        mismatch_idx.append(id_row)
        swapped_qids.append({'id': id_row, 'not_in_cidr':not_in_cidr, 'not_in_iclr':not_in_iclr})
        print(row_cidr['sentence'])
        print("ICLR:", [es_0905.get_title(qid) for qid in not_in_cidr])
        print("ICLR_FULL:", not_in_cidr_spans)
        print("CIDR", [es.get_title(qid) for qid in not_in_iclr])
        print("CIDR_FULL:", not_in_iclr_spans)
        print()
    
    all_iclr_qid.extend(row_iclr['qids'].item())
    all_cidr_qid.extend(row_cidr['qids'])
    
    if ind == 10:
        break
           
print(len(mismatch_idx), 'examples have mismtatches in the QIDs')

0it [00:00, ?it/s]/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: `item` has been deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/

Tom Thabane resigned in October last year to form the All Basotho Convention ( ABC ) , crossing the floor with 17 members of parliament , causing constitutional monarch King Letsie III to dissolve parliament and call the snap election .
ICLR: ['Member of parliament']
ICLR_FULL: [[0, 2], [10, 13], [14, 15], [17, 20], [22, 25], [27, 29], [29, 32], [33, 35], [38, 40]]
CIDR ['New Zealand Parliament']
CIDR_FULL: []

In 1983 , a year after the rally , Forsberg received the so-called `` genius award '' from the John D. and Catherine T. MacArthur Foundation .
ICLR: ['Rallying']
ICLR_FULL: [[7, 8], [9, 10], [14, 16], [19, 26]]
CIDR []
CIDR_FULL: []

This was among a batch of paperback Oxford World 's Classics that I was given as a reward for reading and commenting on a manuscript for OUP .
ICLR: ['Batch file', 'University of Reading']
ICLR_FULL: [[4, 5], [7, 11], [17, 18], [19, 20], [26, 27]]
CIDR ['Batch production', 'Reading comprehension']
CIDR_FULL: []

The latest investigation was authoriz

In [78]:
# visualize the model performance with the same spans for both 

# test_df = bootleg_labels_df[bootleg_labels_df['id'].isin(test_ids)]
test_df = bootleg_labels_df[bootleg_labels_df['id'].isin(mismatch_subjobj_error_id)]

for ind, row_cidr in tqdm(test_df.iterrows()):
    id_row = row_cidr['id']
    row_iclr = bootleg_labels_df_0905[bootleg_labels_df_0905['id']==id_row]
    
    cidr_spans = [get_span(row_cidr['spans'][i]) for i in range(len(row_cidr['spans']))]
    cidr_idx = [i for i in range(len(cidr_spans)) if cidr_spans[i] in row_iclr['spans'].item()]
    iclr_idx = [i for i in range(len(row_iclr['spans'].item())) if row_iclr['spans'].item()[i] in cidr_spans]
    in_cidr_qid = [row_cidr['qids'][i] for i in cidr_idx]
    in_cidr_conf = [row_cidr['probs'][i] for i in cidr_idx]
    in_iclr_qid = [row_iclr['qids'].item()[i] for i in iclr_idx]
    in_iclr_conf = [row_iclr['probs'].item()[i] for i in iclr_idx]
    
    mismatch = []
    for x,y,z,w in zip(in_cidr_qid, in_iclr_qid, in_cidr_conf, in_iclr_conf):
#         if x != y:
        mismatch.append((x,y,z,w))
            
    cidr_pred = df_cidr[df_cidr['id'] == id_row]['prediction'].item()
    iclr_pred = df_iclr[df_iclr['id'] == id_row]['prediction'].item()
    gold_rel = df_iclr[df_iclr['id'] == id_row]['relation'].item()
    subj = df_iclr[df_iclr['id'] == id_row]['subj'].item()
    obj = df_iclr[df_iclr['id'] == id_row]['obj'].item()
        
    if mismatch and (cidr_pred != gold_rel or iclr_pred != gold_rel):
        print(row_cidr['sentence'])
        print('GOLD: {}, SUBJ: {}, OBJ: {}'.format(gold_rel, subj, obj))
        print('----------------------------------------------------------------------')
        print("CIDR PRED: {}, ICLR PRED: {}".format(cidr_pred, iclr_pred))
        for mis in mismatch:
            print("{} ({:.2f}) / {} ({:.2f})".format(es.get_title(mis[0]), mis[2], es_0905.get_title(mis[1]), mis[3]))
        print()
        print()


0it [00:00, ?it/s]/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: `item` has been deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
/dfs/scratch1/simran/tutorial/bootleg-inter

He has served as a policy aide to the late U.S. Senator Alan Cranston , as National Issues Director for the 2004 presidential campaign of Congressman Dennis Kucinich , as a co-founder of Progressive Democrats of America and as a member of the international policy department at the RAND Corporation think tank before all that .
GOLD: no_relation, SUBJ: ['progressive', 'democrats', 'of', 'america'], OBJ: ['international', 'policy', 'department']
----------------------------------------------------------------------
CIDR PRED: no_relation, ICLR PRED: per:employee_of
Aide Ganasi (0.40) / Aide-de-camp (0.93)
Alan Cranston (1.00) / Alan Cranston (1.00)
2004 United States presidential election (0.51) / 2004 United States presidential election (0.96)
United States House of Representatives (0.52) / United States House of Representatives (0.85)
Dennis Kucinich (1.00) / Dennis Kucinich (1.00)
Progressive Democrats of America (1.00) / Progressive Democrats of America (1.00)
Ministry (government dep

20it [00:00, 40.74it/s]

He said that according to his investigation , Bibi drew the ire of fellow farmhands after a dispute in June 2009 , when they refused to drink water she collected and she refused their demands that she convert to Islam .
GOLD: no_relation, SUBJ: ['bibi'], OBJ: ['islam']
----------------------------------------------------------------------
CIDR PRED: no_relation, ICLR PRED: per:religion
Taramon Bibi (0.36) / Mukhtar Mai (0.28)
Draft (hull) (0.67) / Tie (draw) (0.41)
Iron response element (0.61) / Ira (Messenia) (0.50)
Religious conversion (0.75) / Religious conversion (0.79)


A spokesman for Countrywide , Rick Simon , said on Wednesday night that `` as a matter of policy , we do n't comment on communications with regulators . ''
GOLD: org:top_members/employees, SUBJ: ['countrywide'], OBJ: ['rick', 'simon']
----------------------------------------------------------------------
CIDR PRED: org:top_members/employees, ICLR PRED: no_relation
Bank of America Home Loans (0.68) / Countrywide (0

32it [00:00, 46.01it/s]

...

He is also survived by his parents and a sister , Karen Lange , of Washington , and a brother , Adam Lange , of St. Louis .
GOLD: no_relation, SUBJ: ['adam', 'lange'], OBJ: ['his']
----------------------------------------------------------------------
CIDR PRED: no_relation, ICLR PRED: per:siblings
Jackie Speier (0.52) / Karen Carpenter (0.99)
Oskar R. Lange (0.61) / Michael Lange (0.99)
Washington, D.C. (0.97) / Washington, D.C. (0.98)
Karl Kehrle (1.00) / Karl Kehrle (1.00)
Oskar R. Lange (0.55) / Michael Lange (0.99)
St. Louis (0.99) / St. Louis (0.99)


Services for 61-year-old Carol Daniels will be Monday at 1 p.m. local time in Oklahoma City .
GOLD: per:age, SUBJ: ['carol', 'daniels'], OBJ: ['61-year-old']
----------------------------------------------------------------------
CIDR PRED: no_relation, ICLR PRED: per:age
Hurricane Carol (0.36) / Christmas carol (0.43)
Patricia Cornwell (0.26) / Mitch Daniels (0.84)
Eastern Time Zone (0.33) / Eastern Time Zone (0.50)
Oklahoma City (0

383it [00:05, 68.60it/s]

The body of 39-year-old Joseph Cerniglia , whose restaurant experienced a renewal since the show , was found floating in the river on Friday .
GOLD: no_relation, SUBJ: ['joseph', 'cerniglia'], OBJ: ['friday']
----------------------------------------------------------------------
CIDR PRED: no_relation, ICLR PRED: per:date_of_death
Saint Joseph (0.93) / Pope Benedict XVI (0.30)
Friday the 13th (0.67) / Good Friday (0.80)


Two of Jupp 's managers at the Defense , Scientific and Technology Laboratory were charged with manslaughter following an inquiry into the blast .
GOLD: no_relation, SUBJ: ['jupp'], OBJ: ['defense']
----------------------------------------------------------------------
CIDR PRED: per:employee_of, ICLR PRED: no_relation
Alex Jupp (0.39) / James Jupp (0.36)
Manslaughter in English law (0.57) / Manslaughter (United States law) (0.47)
Explosion (0.31) / Explosion (0.54)


`` We have been strong advocates and supporters in trying to ensure this provision became law , and a

396it [00:06, 65.13it/s]

Some 200 workers at the New Fabris factory outside the southwest city of Chatellerault , are each demanding euro30 ,000 ( $ 42,267 ) from Renault and PSA Peugeot-Citroen , accusing the carmakers of killing their livelihoods .
GOLD: org:city_of_headquarters, SUBJ: ['new', 'fabris'], OBJ: ['chatellerault']
----------------------------------------------------------------------
CIDR PRED: org:city_of_headquarters, ICLR PRED: no_relation
Stanko Fabris (0.53) / Fabris (0.99)
Southwest City, Missouri (1.00) / Southwest City, Missouri (1.00)
Châtellerault (0.93) / Châtellerault (0.99)
Renault (0.98) / Renault (0.97)
Groupe PSA (0.94) / Groupe PSA (0.98)


`` But I would be wary of saying he was behind the kidnapping and murder of the Polish engineer , '' he added .
GOLD: no_relation, SUBJ: ['he'], OBJ: ['kidnapping']
----------------------------------------------------------------------
CIDR PRED: per:charges, ICLR PRED: no_relation
Kidnapping of Aldo Moro (0.59) / Lindbergh kidnapping (0.84)


In [32]:
df_cidr.columns

Index(['obj', 'subj', 'prop_ner', 'obj_pos', 'subj_ner', 'obj_mentions',
       'subj_type', 'obj_ner', 'id', 'subj_pos', 'prediction', 'obj_qids',
       'example', 'qids', 'subj_qids', 'prop_mentions', 'num_ner', 'obj_type',
       'obj_leng', 'subj_leng', 'relation', 'real_mentions', 'stanford_ner',
       'subj_mentions', 'separation_dist', 'mentions'],
      dtype='object')

In [41]:
# Count: number of qids per model, number of total mismatches, and number of total mismatch

iclr_qids = []
iclr_qids_subjobj = []
cidr_qids = []
cidr_qids_subjobj = []

mismatch_all = []
mismatch_subjobj = []

qids_cidr_prob_1 = []
qids_iclr_prob_1 = []

for ind, row_cidr in tqdm(test_df.iterrows()):
    id_row = row_cidr['id']
    row_iclr = bootleg_labels_df_0905[bootleg_labels_df_0905['id']==id_row]
    
    # total
    cidr_qids.extend(row_cidr['qids'])
    iclr_qids.extend(row_iclr['qids'].item())
    
    # total subj obj
    iclr_subj = df_iclr[df_iclr['id'] == id_row]['subj_qids'].item()
    iclr_subj = ast.literal_eval(iclr_subj)
    iclr_obj = df_iclr[df_iclr['id'] == id_row]['obj_qids'].item()
    iclr_obj = ast.literal_eval(iclr_obj)
    iclr_qids_subjobj.extend(iclr_subj)
    iclr_qids_subjobj.extend(iclr_obj)
    
    cidr_subj = df_cidr[df_iclr['id'] == id_row]['subj_qids'].item()
    cidr_subj = ast.literal_eval(cidr_subj)
    cidr_obj = df_cidr[df_iclr['id'] == id_row]['obj_qids'].item()
    cidr_obj = ast.literal_eval(cidr_obj)
    cidr_qids_subjobj.extend(cidr_subj)
    cidr_qids_subjobj.extend(cidr_obj)
    
    # mismatches subj obj
    if any(qid not in cidr_subj for qid in iclr_subj) or any(qid not in cidr_obj for qid in iclr_obj):
        mismatch_subjobj.append({'subj': (iclr_subj, cidr_subj), 'obj':(iclr_obj, cidr_obj)})
        
        
    cidr_spans = [get_span(row_cidr['spans'][i]) for i in range(len(row_cidr['spans']))]
    cidr_idx = [i for i in range(len(cidr_spans)) if cidr_spans[i] in row_iclr['spans'].item()]
    iclr_idx = [i for i in range(len(row_iclr['spans'].item())) if row_iclr['spans'].item()[i] in cidr_spans]
    in_cidr_qid = [row_cidr['qids'][i] for i in cidr_idx]
    in_cidr_conf = [row_cidr['probs'][i] for i in cidr_idx]
    in_iclr_qid = [row_iclr['qids'].item()[i] for i in iclr_idx]
    in_iclr_conf = [row_iclr['probs'].item()[i] for i in iclr_idx]
    
    for x,y in zip(in_cidr_qid, in_iclr_qid):
        if x != y:
            mismatch_all.append((x,y))
    
    for x,y in zip(in_cidr_qid, in_cidr_conf):
        if y == 1.00:
            qids_cidr_prob_1.append(x)
        
    for x,y in zip(in_iclr_qid, in_iclr_conf):
        if y == 1.00:
            qids_iclr_prob_1.append(x)


0it [00:00, ?it/s]/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: `item` has been deprecated and will be removed in a future version
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: `item` has been deprecated and will be removed in a future version
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:25: FutureWarning: `item` has been deprecated and will be removed in a future version
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: `item` has been deprecated and will be removed in a future version
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:32: FutureWarning: `item` has been deprecated and will be removed in a future versi

In [77]:
# repeat for error examples only
all_error_ids = cidr_error_ids
all_error_ids.extend(iclr_error_ids)
all_error_ids = set(all_error_ids)
test_error_df = bootleg_labels_df[bootleg_labels_df['id'].isin(all_error_ids)]
print(test_error_df.shape[0])

iclr_qids_error = []
iclr_qids_subjobj_error = []
cidr_qids_error = []
cidr_qids_subjobj_error = []

mismatch_all_error = []
mismatch_subjobj_error = []
mismatch_subjobj_error_id = []

qids_cidr_prob_1_error = []
qids_iclr_prob_1_error = []

for ind, row_cidr in tqdm(test_error_df.iterrows()):
    id_row = row_cidr['id']
    row_iclr = bootleg_labels_df_0905[bootleg_labels_df_0905['id']==id_row]
    
    # total
    cidr_qids_error.extend(row_cidr['qids'])
    iclr_qids_error.extend(row_iclr['qids'].item())
    
    # total subj obj
    iclr_subj = df_iclr[df_iclr['id'] == id_row]['subj_qids'].item()
    iclr_subj = ast.literal_eval(iclr_subj)
    iclr_obj = df_iclr[df_iclr['id'] == id_row]['obj_qids'].item()
    iclr_obj = ast.literal_eval(iclr_obj)
    iclr_qids_subjobj_error.extend(iclr_subj)
    iclr_qids_subjobj_error.extend(iclr_obj)
    
    cidr_subj = df_cidr[df_cidr['id'] == id_row]['subj_qids'].item()
    cidr_subj = ast.literal_eval(cidr_subj)
    cidr_obj = df_cidr[df_cidr['id'] == id_row]['obj_qids'].item()
    cidr_obj = ast.literal_eval(cidr_obj)
    cidr_qids_subjobj_error.extend(cidr_subj)
    cidr_qids_subjobj_error.extend(cidr_obj)
    
    
    gold_relation = df_cidr[df_cidr['id'] == id_row]['relation'].item()
    cidr_prediction = df_cidr[df_cidr['id'] == id_row]['prediction'].item()
    iclr_prediction = df_iclr[df_iclr['id'] == id_row]['prediction'].item()
    # mismatches subj obj
    if any(qid not in cidr_subj for qid in iclr_subj) or any(qid not in cidr_obj for qid in iclr_obj):
        mismatch_subjobj_error.append({'subj': (iclr_subj, cidr_subj), 'obj':(iclr_obj, cidr_obj)})
        if not (iclr_prediction != gold_relation and cidr_prediction != gold_relation):
            mismatch_subjobj_error_id.append(id_row)
        
    cidr_spans = [get_span(row_cidr['spans'][i]) for i in range(len(row_cidr['spans']))]
    cidr_idx = [i for i in range(len(cidr_spans)) if cidr_spans[i] in row_iclr['spans'].item()]
    iclr_idx = [i for i in range(len(row_iclr['spans'].item())) if row_iclr['spans'].item()[i] in cidr_spans]
    in_cidr_qid = [row_cidr['qids'][i] for i in cidr_idx]
    in_cidr_conf = [row_cidr['probs'][i] for i in cidr_idx]
    in_iclr_qid = [row_iclr['qids'].item()[i] for i in iclr_idx]
    in_iclr_conf = [row_iclr['probs'].item()[i] for i in iclr_idx]
    
    for x,y in zip(in_cidr_qid, in_iclr_qid):
        if x != y:
            mismatch_all_error.append((x,y))
    
    for x,y in zip(in_cidr_qid, in_cidr_conf):
        if y == 1.00:
            qids_cidr_prob_1_error.append(x)
        
    for x,y in zip(in_iclr_qid, in_iclr_conf):
        if y == 1.00:
            qids_iclr_prob_1_error.append(x)
    

0it [00:00, ?it/s]/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: `item` has been deprecated and will be removed in a future version
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: `item` has been deprecated and will be removed in a future version
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: `item` has been deprecated and will be removed in a future version
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: `item` has been deprecated and will be removed in a future version
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:38: FutureWarning: `item` has been deprecated and will be removed in a future versi

1639


1639it [00:26, 61.89it/s]


In [73]:
print(len(mismatch_subjobj_error_id))

396


In [67]:
print("ERRORS ON TEST!")
print("Number of ICLR QIDs total: {}".format(len(iclr_qids_error)))
print("Number of CIDR QIDs total: {}".format(len(cidr_qids_error)))
print("Number of ICLR QIDs total SET: {}".format(len(set(iclr_qids_error))))
print("Number of CIDR QIDs total SET: {}\n".format(len(set(cidr_qids_error))))

print("Number of ICLR QIDs subjobj: {}".format(len(iclr_qids_subjobj_error)))
print("Number of CIDR QIDs subjobj: {}".format(len(cidr_qids_subjobj_error)))
print("Number of ICLR QIDs subjobj SET: {}".format(len(set(iclr_qids_subjobj_error))))
print("Number of CIDR QIDs subjobj SET: {}\n".format(len(set(cidr_qids_subjobj_error))))

print("Number of Mismatches ALL (on places with matching span): {}".format(len(mismatch_all_error)))
print("Number of Mismatches SubjObj: {}\n".format(len(mismatch_subjobj_error)))

print("Number of ICLR QIDs with prob 1: {}".format(len(qids_iclr_prob_1_error)))
print("Number of CIDR QIDs with prob 1: {}".format(len(qids_cidr_prob_1_error)))
print("Number of ICLR QIDs with prob 1 SET: {}".format(len(set(qids_iclr_prob_1_error))))
print("Number of CIDR QIDs with prob 1 SET: {}\n".format(len(set(qids_cidr_prob_1_error))))


ERRORS ON TEST!
Number of ICLR QIDs total: 11211
Number of CIDR QIDs total: 9929
Number of ICLR QIDs total SET: 3451
Number of CIDR QIDs total SET: 3181

Number of ICLR QIDs subjobj: 5460
Number of CIDR QIDs subjobj: 5460
Number of ICLR QIDs subjobj SET: 910
Number of CIDR QIDs subjobj SET: 887

Number of Mismatches ALL (on places with matching span): 2614
Number of Mismatches SubjObj: 656

Number of ICLR QIDs with prob 1: 2085
Number of CIDR QIDs with prob 1: 2036
Number of ICLR QIDs with prob 1 SET: 665
Number of CIDR QIDs with prob 1 SET: 652



In [56]:
print("OVERALL TEST")
print("Number of ICLR QIDs total: {}".format(len(iclr_qids)))
print("Number of CIDR QIDs total: {}".format(len(cidr_qids)))
print("Number of ICLR QIDs total SET: {}".format(len(set(iclr_qids))))
print("Number of CIDR QIDs total SET: {}\n".format(len(set(cidr_qids))))

print("Number of ICLR QIDs subjobj: {}".format(len(iclr_qids_subjobj)))
print("Number of CIDR QIDs subjobj: {}".format(len(cidr_qids_subjobj)))
print("Number of ICLR QIDs subjobj SET: {}".format(len(set(iclr_qids_subjobj))))
print("Number of CIDR QIDs subjobj SET: {}\n".format(len(set(cidr_qids_subjobj))))

print("Number of Mismatches ALL (on places with matching span): {}".format(len(mismatch_all)))
print("Number of Mismatches SubjObj: {}\n".format(len(mismatch_subjobj)))

print("Number of ICLR QIDs with prob 1: {}".format(len(qids_iclr_prob_1)))
print("Number of CIDR QIDs with prob 1: {}".format(len(qids_cidr_prob_1)))
print("Number of ICLR QIDs with prob 1 SET: {}".format(len(set(qids_iclr_prob_1))))
print("Number of CIDR QIDs with prob 1 SET: {}\n".format(len(set(qids_cidr_prob_1))))

OVERALL TEST
Number of ICLR QIDs total: 100188
Number of CIDR QIDs total: 88216
Number of ICLR QIDs total SET: 11473
Number of CIDR QIDs total SET: 10532

Number of ICLR QIDs subjobj: 48947
Number of CIDR QIDs subjobj: 48947
Number of ICLR QIDs subjobj SET: 4242
Number of CIDR QIDs subjobj SET: 4129

Number of Mismatches ALL (on places with matching span): 23706
Number of Mismatches SubjObj: 5389

Number of ICLR QIDs with prob 1: 18945
Number of CIDR QIDs with prob 1: 18822
Number of ICLR QIDs with prob 1 SET: 2470
Number of CIDR QIDs with prob 1 SET: 2460



In [68]:
# for mismatched qids, which model is better?

for item in mismatch_subjobj_error:
    subj = item['subj']
    
    
    obj = item['obj']
    break


{'subj': (['Q4379875', 'Q4379875', 'Q4379875', 'Q4379875'], ['Q4379875', 'Q4379875', 'Q4379875', 'Q4379875']), 'obj': (['UNK', 'UNK', 'Q2467461'], ['UNK', 'UNK', 'Q192350'])}


# Generate a dataset that uses the CIDR spans, but ICLR QIDs

In [25]:
def convert_span_format(str_span):
    #'11:14'
    ss, se = str_span.split(":")
    ss = int(ss)
    se = int(se)
    return [ss,se]

In [26]:
df_final = bootleg_labels_df.copy()

for ind, row_cidr in tqdm(bootleg_labels_df.iterrows()):
    id_row = row_cidr['id']
    row_iclr = bootleg_labels_df_0905[bootleg_labels_df_0905['id']==id_row]
    
    cidr_spans = [convert_span_format(row_cidr['spans'][i]) for i in range(len(row_cidr['spans']))]
    cidr_spans_filt = [span for span in cidr_spans if span in row_iclr['spans'].item()]

    iclr_indices = [row_iclr['spans'].item().index(span) for span in cidr_spans if span in row_iclr['spans'].item()]
    
    iclr_aliases = [row_iclr['aliases'].item()[i] for i in iclr_indices]
    iclr_probs = [row_iclr['probs'].item()[i] for i in iclr_indices]
    iclr_ctx_emb_ids = [row_iclr['ctx_emb_ids'].item()[i] for i in iclr_indices]
    iclr_entity_ids = [row_iclr['entity_ids'].item()[i] for i in iclr_indices]
    iclr_qids = [row_iclr['qids'].item()[i] for i in iclr_indices]
    
    df_final.at[ind, 'spans'] = cidr_spans_filt
    df_final.at[ind, 'aliases'] = iclr_aliases
    df_final.at[ind, 'probs'] = iclr_probs
    df_final.at[ind, 'ctx_emb_ids'] = iclr_ctx_emb_ids
    df_final.at[ind, 'entity_ids'] = iclr_entity_ids
    df_final.at[ind, 'qids'] = iclr_qids
    
#     print("CIDR SPANS:", cidr_spans_filt)
#     print("ICLR SPANS:", row_iclr['spans'].item())
#     print("ICLR QIDS (FULL)", row_iclr['qids'].item())
#     print("ICLR INDICES:", iclr_indices)
#     print("ICLR QIDS (FILT):", iclr_qids)
#     print("")
#     if ind == 5:
#         break

0it [00:00, ?it/s]/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: `item` has been deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykern

In [27]:
# df_final[0:6]

In [28]:
base_directory_0905 = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_092620/'
expt_directory_0905 = 'basic_full_sentences/'
out_file = '{}{}/cidr_spans_iclr_qids/bootleg_labels.jsonl'.format(base_directory_0905, expt_directory_0905)
# Output in JSONL format
print(df_final.to_json(out_file, orient='records', lines=True))

None


# load tagme preds

In [29]:
def match(orig_label_file, new_label_file, threshold=None):
    # read in first file and map by index for fast retrieval
    total_mentions = 0
    correct_mentions = 0
    pred_mentions = 0

    new_labels = {}
    limit = 5000
    with jsonlines.open(new_label_file) as f:
        for line in f:
            new_labels[line['id']] = line
            limit -= 1
            if limit == 0:
                break

    row_results = {}
    
    with jsonlines.open(orig_label_file) as f:
        for line in f:
            gold_aliases = line['aliases']
            gold_spans = line['spans']
            gold_qids = line['qids']
            idx = line['id']
            if idx in new_labels:
                in_tagme_only = []
                in_bootleg_only = []
                in_both = []
                tagme_qids =new_labels[idx]['qids']
                in_tagme_only = [tag_qid for tag_qid in tagme_qids if tag_qid not in gold_qids]
                in_bootleg_only = [gold_qid for gold_qid in gold_qids if gold_qid not in tagme_qids]
                in_both = [tag_qid for tag_qid in tagme_qids if tag_qid in gold_qids]
                
                entry = {'tagme_qid_only':in_tagme_only, 'in_bootleg_only':in_bootleg_only, 'in_both':in_both, 
                         'tagme_row':new_labels[idx], 'bootleg_row':line}
                row_results[idx] = entry 

    return row_results


In [30]:
import tagme
sys.path.append('/dfs/scratch1/simran/tutorial/bootleg-internal/tutorials/')
from utils import *
# Set the authorization token for subsequent calls.
tagme.GCUBE_TOKEN = '03d48ddc-cad1-4e11-842a-996a851bed41-843339462'

### ICLR

In [31]:
# As the threshold increases, the precision increases, but the recall decreases
# We use a mapping from Wikipedia pageids to Wikidata QIDs to get the QIDs predicted by TAGME 
wpid2qid = ujson.load(open(f'/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_downloads/entity_db/entity_mappings/wpid2qid.json'))
input_dir = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_092620/basic_full_sentences'
in_file = f'{input_dir}/all_tacred_bootinput.jsonl'
tagme_label_file = f'{input_dir}/all_tacred_bootinput_tagme.jsonl'

#tagme_annotate(in_file=in_file, out_file=tagme_label_file, threshold=0.3, wpid2qid=wpid2qid)

In [32]:
in_file_bootleg = f'{input_dir}/bootleg_labels.jsonl'
print(in_file_bootleg)
row_results = match(orig_label_file=in_file_bootleg, new_label_file=tagme_label_file)

/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_092620/basic_full_sentences/bootleg_labels.jsonl


In [33]:
#stop_count = 50
tagme_only = 0
bootleg_only = 0
matches = 0
iclr_bootleg_only_lst = []
iclr_tagme_only_lst = []
iclr_both_lst = []
for k, v in row_results.items():
#    print("{}:\ntagme_qid_only: {}\nin_bootleg_only: {}\nin_both: {}\n".format(k, v['tagme_qid_only'], 
#                                                                               v['in_bootleg_only'], v['in_both']))
    tagme_only += len(v['tagme_qid_only']) 
    bootleg_only += len(v['in_bootleg_only'])
    matches += len(v['in_both'])
    
    iclr_bootleg_only_lst.extend(v['in_bootleg_only'])
    iclr_tagme_only_lst.extend(v['tagme_qid_only'])
    iclr_both_lst.extend(v['in_both'])
    
#     stop_count -= 1
#     if stop_count == 0:
#         break
print("Matches: {}, TAGME_only: {}, BOOTLEG_only:{}".format(matches, tagme_only, bootleg_only))

Matches: 12299, TAGME_only: 3626, BOOTLEG_only:24319


In [34]:
print("BOOT ONLY", len(set(iclr_bootleg_only_lst)))
print("TAGME ONLY", len(set(iclr_tagme_only_lst)))
print("BOTH", len(set(iclr_both_lst)))

BOOT ONLY 10728
TAGME ONLY 1815
BOTH 4563


### CIDR

In [35]:
# As the threshold increases, the precision increases, but the recall decreases
# We use a mapping from Wikipedia pageids to Wikidata QIDs to get the QIDs predicted by TAGME 
wpid2qid_cidr = ujson.load(open(f'/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_cidr_model/bootleg_downloads/entity_db/entity_mappings/wpid2qid.json'))
input_dir_cidr = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_cidr_model/bootleg_09132020/basic_full_sentences'
in_file_cidr = f'{input_dir_cidr}/all_tacred_bootinput.jsonl'
tagme_label_file_cidr = f'{input_dir_cidr}/all_tacred_bootinput_tagme.jsonl'

#tagme_annotate(in_file=in_file_cidr, out_file=tagme_label_file_cidr, threshold=0.3, wpid2qid=wpid2qid)

In [36]:
in_file_bootleg_cidr = f'{input_dir_cidr}/bootleg_labels.jsonl'
print(in_file_bootleg_cidr)
row_results_cidr = match(orig_label_file=in_file_bootleg_cidr, new_label_file=tagme_label_file_cidr)

/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_cidr_model/bootleg_09132020/basic_full_sentences/bootleg_labels.jsonl


In [37]:
stop_count = 50

cidr_tagme_only = 0
cidr_bootleg_only = 0
cidr_matches = 0
cidr_tagme_only_lst = []
cidr_both_lst = []
cidr_bootleg_only_lst = []
for k, v in row_results_cidr.items():
    #print("{}:\ntagme_qid_only: {}\nin_bootleg_only: {}\nin_both: {}\n".format(k, v['tagme_qid_only'], 
    #                                                                            v['in_bootleg_only'], v['in_both']))
    
    cidr_matches += len(v['in_both'])
    cidr_tagme_only += len(v['tagme_qid_only'])
    cidr_bootleg_only += len(v['in_bootleg_only'])
    
    cidr_both_lst.extend(v['in_both'])
    cidr_bootleg_only_lst.extend(v['in_bootleg_only'])
    cidr_tagme_only_lst.extend(v['tagme_qid_only'])
    

print("Matches: {}, TAGME_only: {}, BOOTLEG_only:{}".format(cidr_matches, cidr_tagme_only, cidr_bootleg_only))


Matches: 12234, TAGME_only: 3687, BOOTLEG_only:20357


In [38]:
print("BOOT ONLY", len(set(cidr_bootleg_only_lst)))
print("TAGME ONLY", len(set(cidr_tagme_only_lst)))
print("BOTH", len(set(cidr_both_lst)))

BOOT ONLY 9742
TAGME ONLY 1894
BOTH 4526


### Names are mismatched (oops)

In [ ]:
# this means that ICLR adds these QIDs which CIDR does not use
unique_to_iclr_boot = [qid_ for qid_ in cidr_bootleg_only_lst if qid_ not in iclr_bootleg_only]
unique_to_iclr_tagme = [qid_ for qid_ in cidr_tagme_only_lst if qid_ not in iclr_tagme_only_lst]
unique_to_iclr_both = [qid_ for qid_ in cidr_both_lst if qid_ not in iclr_both_lst]

In [ ]:
print(len(set(unique_to_iclr_boot)))
print(len(set(unique_to_iclr_tagme)))
print(len(set(unique_to_iclr_both)))

In [ ]:
# this means that cidr has a large amount different, even amongst the fewer that it gets in the first place. Also
# ICLr provides more mentions
unique_to_cidr_boot = [qid_ for qid_ in iclr_bootleg_only if qid_ not in cidr_bootleg_only_lst]
unique_to_cidr_tagme = [qid_ for qid_ in iclr_tagme_only_lst if qid_ not in cidr_tagme_only_lst]
unique_to_cidr_both = [qid_ for qid_ in iclr_both_lst if qid_ not in cidr_both_lst]

In [ ]:
print(len(set(unique_to_cidr_boot)))
print(len(set(unique_to_cidr_tagme)))
print(len(set(unique_to_cidr_both)))